## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zlobin,HR,45.2924,14.6541,43.38,84,75,5.75,broken clouds
1,1,Yaqui,MX,27.3667,-110.1667,58.91,59,0,11.52,clear sky
2,2,Kodiak,US,57.7900,-152.4072,36.91,48,100,5.75,overcast clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,63.23,68,5,10.47,clear sky
4,4,Busselton,AU,-33.6500,115.3333,68.04,56,65,20.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <=max_temp)&
                                    (city_data_df["Max Temp"]>= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ponta Do Sol,PT,32.6667,-17.1000,63.23,68,5,10.47,clear sky
4,4,Busselton,AU,-33.6500,115.3333,68.04,56,65,20.00,broken clouds
5,5,Kavaratti,IN,10.5669,72.6420,77.52,67,16,6.17,few clouds
6,6,Souillac,MU,-20.5167,57.5167,77.38,94,75,3.44,broken clouds
7,7,New Norfolk,AU,-42.7826,147.0587,77.88,30,0,1.99,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ponta Do Sol,PT,32.6667,-17.1000,63.23,68,5,10.47,clear sky
4,4,Busselton,AU,-33.6500,115.3333,68.04,56,65,20.00,broken clouds
5,5,Kavaratti,IN,10.5669,72.6420,77.52,67,16,6.17,few clouds
6,6,Souillac,MU,-20.5167,57.5167,77.38,94,75,3.44,broken clouds
7,7,New Norfolk,AU,-42.7826,147.0587,77.88,30,0,1.99,clear sky
...,...,...,...,...,...,...,...,...,...,...
643,643,Ippy,CF,6.2679,21.2247,69.28,18,65,7.85,broken clouds
645,645,Carutapera,BR,-1.1950,-46.0200,75.67,94,100,4.45,overcast clouds
648,648,Phan Thiet,VN,10.9333,108.1000,85.98,55,70,11.07,broken clouds
657,657,Samana,DO,19.2056,-69.3369,71.98,88,23,9.60,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ponta Do Sol,PT,63.23,clear sky,32.6667,-17.1000,
4,Busselton,AU,68.04,broken clouds,-33.6500,115.3333,
5,Kavaratti,IN,77.52,few clouds,10.5669,72.6420,
6,Souillac,MU,77.38,broken clouds,-20.5167,57.5167,
7,New Norfolk,AU,77.88,clear sky,-42.7826,147.0587,
8,Venice,US,60.98,mist,27.0998,-82.4543,
9,Rikitea,PF,79.00,few clouds,-23.1203,-134.9692,
11,Bac Lieu,VN,83.53,overcast clouds,9.2850,105.7244,
12,Ushuaia,AR,60.46,broken clouds,-54.8000,-68.3000,
13,Alofi,NU,82.29,overcast clouds,-19.0595,-169.9187,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
output_data_file = "WeatherPy_vacation.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))